<a href="https://colab.research.google.com/github/lwx-3000/Deep_Neuron_Network/blob/main/simplified_banking_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplified Banking System

This project is a Python implementation of a simplified banking system. It is an object-oriented program that simulates basic features of a banking application, including account creation, deposits, transfers, payments, and account merging.


## Features

- **Level 1**: Account creation, deposits, and money transfers between accounts.
- **Level 2**: Ranking accounts based on the total amount of outgoing transactions.
- **Level 3**: Scheduling payments with cashback and checking the status of scheduled payments.
- **Level 4**: (Planned) Merging two accounts while retaining balance and transaction histories.


Level 1

In [ ]:
class Account:
    def __init__(self, account_id, balance=0):
        self.account_id = account_id
        self.balance = balance

class Bank:
    def __init__(self):
        self.accounts = {}  # Maps account_id to Account object

    def create_account(self, timestamp, account_id):
        if account_id in self.accounts:
            return "false"
        self.accounts[account_id] = Account(account_id)
        return "true"

    def deposit(self, timestamp, account_id, amount):
        if account_id not in self.accounts:
            return ""
        account = self.accounts[account_id]
        account.balance += amount
        return str(account.balance)

    def transfer(self, timestamp, source_account_id, target_account_id, amount):
        if source_account_id == target_account_id or source_account_id not in self.accounts or target_account_id not in self.accounts:
            return ""
        source_account = self.accounts[source_account_id]
        target_account = self.accounts[target_account_id]
        if source_account.balance < amount:
            return ""
        source_account.balance -= amount
        target_account.balance += amount
        return str(source_account.balance)



Level 2

In [ ]:
class Account:
    def __init__(self, account_id, balance=0):
        self.account_id = account_id
        self.balance = balance
        self.outgoing_transactions = 0  # Total amount transferred out

class Bank(Bank):
    # Existing methods remain unchanged

    def transfer(self, timestamp, source_account_id, target_account_id, amount):
        if source_account_id == target_account_id or source_account_id not in self.accounts or target_account_id not in self.accounts:
            return ""
        source_account = self.accounts[source_account_id]
        target_account = self.accounts[target_account_id]
        if source_account.balance < amount:
            return ""
        source_account.balance -= amount
        target_account.balance += amount
        source_account.outgoing_transactions += amount  # Track outgoing transaction
        return str(source_account.balance)

    def top_spenders(self, timestamp, n):
        # Sort accounts by outgoing transactions in descending order, then alphabetically
        ranked_accounts = sorted(self.accounts.values(), key=lambda x: (-x.outgoing_transactions, x.account_id))
        # Format output
        result = ", ".join(f"{account.account_id}({account.outgoing_transactions})" for account in ranked_accounts[:n])
        return result



Level 3

In [ ]:

class Account:
    def __init__(self, account_id):
        self.account_id = account_id
        self.balance = 0
        self.outgoing_transactions = 0  # For Level 2
        self.scheduled_payments = []  # For Level 3

    def make_payment(self, amount, timestamp):
        payment_id = f"payment{len(self.scheduled_payments) + 1}"
        cashback_amount = int(amount * 0.02)  # 2% cashback, rounded down
        cashback_timestamp = int(timestamp) + 86400000  # 24 hours later
        self.balance -= amount
        self.outgoing_transactions += amount
        self.scheduled_payments.append((payment_id, cashback_amount, cashback_timestamp, "IN_PROGRESS"))
        return payment_id

    def process_cashbacks(self, current_timestamp):
        for i, (payment_id, cashback_amount, cashback_timestamp, status) in enumerate(self.scheduled_payments):
            if current_timestamp >= cashback_timestamp and status == "IN_PROGRESS":
                self.balance += cashback_amount  # Apply cashback
                # Update the payment status directly in the list
                self.scheduled_payments[i] = (payment_id, cashback_amount, cashback_timestamp, "CASHBACK_RECEIVED")

    def get_payment_status(self, payment_id):
        for pid, _, _, status in self.scheduled_payments:
            if pid == payment_id:
                return status
        return ""

class Bank:
    def __init__(self):
        self.accounts = {}

    def create_account(self, timestamp, account_id):
        if account_id in self.accounts:
            return "false"
        self.accounts[account_id] = Account(account_id)
        return "true"

    def deposit(self, timestamp, account_id, amount):
        if account_id not in self.accounts:
            return ""
        self.accounts[account_id].balance += amount
        return str(self.accounts[account_id].balance)

    def transfer(self, timestamp, source_account_id, target_account_id, amount):
        if source_account_id not in self.accounts or target_account_id not in self.accounts or source_account_id == target_account_id or self.accounts[source_account_id].balance < amount:
            return ""
        self.accounts[source_account_id].balance -= amount
        self.accounts[target_account_id].balance += amount
        self.accounts[source_account_id].outgoing_transactions += amount
        return str(self.accounts[source_account_id].balance)

    def top_spenders(self, timestamp, n):
        # Sort accounts by outgoing transactions in descending order, then alphabetically
        ranked_accounts = sorted(self.accounts.values(), key=lambda x: (-x.outgoing_transactions, x.account_id))
        # Format output
        result = ", ".join(f"{account.account_id}({account.outgoing_transactions})" for account in ranked_accounts[:n])
        return result

    def pay(self, timestamp, account_id, amount):
        if account_id not in self.accounts or self.accounts[account_id].balance < amount:
            return ""
        return self.accounts[account_id].make_payment(amount, timestamp)

    def get_payment_status(self, timestamp, account_id, payment_id):
        if account_id not in self.accounts:
            return ""
        return self.accounts[account_id].get_payment_status(payment_id)

    def process_all_cashbacks(self, current_timestamp):
        for account in self.accounts.values():
            account.process_cashbacks(current_timestamp)




Level 4

In [ ]:
class Account(Account):

    def merge_account(self, other_account):
        # Add other account's balance to this one
        self.balance += other_account.balance
        # Merge outgoing transactions
        self.outgoing_transactions += other_account.outgoing_transactions
        # Merge scheduled payments and ensure payment IDs remain unique
        self.scheduled_payments.extend(other_account.scheduled_payments)

class Bank(Bank):
    def merge_accounts(self, timestamp, account_id1, account_id2):
        # Check if both accounts exist and are different
        if account_id1 == account_id2 or account_id1 not in self.accounts or account_id2 not in self.accounts:
            return "false"

        # Merge account2 into account1
        self.accounts[account_id1].merge_account(self.accounts[account_id2])

        # Remove account2 from the system
        del self.accounts[account_id2]

        return "true"

    def get_balance(self, timestamp, account_id):
        # Check if the account exists
        if account_id in self.accounts:
            # Return the current balance of the account
            return str(self.accounts[account_id].balance)
        else:
            # If the account doesn't exist, return an empty string
            return ""


In [ ]:
def process_queries(queries):
    bank = Bank()
    results = []

    for query in queries:
        operation, timestamp_str, *params = query
        timestamp = int(timestamp_str)
        bank.process_all_cashbacks(timestamp)

        if operation == "CREATE_ACCOUNT":
            _, _, account_id = query
            result = bank.create_account(timestamp, account_id)
        elif operation == "PAY":
            _, _, account_id, amount = query
            result = bank.pay(timestamp, account_id, int(amount))
        elif operation == "GET_PAYMENT_STATUS":
            _, _, account_id, payment_id = query
            result = bank.get_payment_status(timestamp, account_id, payment_id)
        elif operation == "TOP_SPENDERS":
            _, _, n = query
            result = bank.top_spenders(timestamp, int(n))
        elif operation == "DEPOSIT":
            _, _, account_id, amount = query
            result = bank.deposit(timestamp, account_id, int(amount))
        elif operation == "TRANSFER":
            _, _, source_account_id, target_account_id, amount = query
            result = bank.transfer(timestamp, source_account_id, target_account_id, int(amount))
        elif operation == "MERGE_ACCOUNTS":
            _, _, account_id1, account_id2 = query
            result = bank.merge_accounts(timestamp, account_id1, account_id2)
        elif operation == "GET_BALANCE":
            _, _, account_id, _ = query
            result = bank.get_balance(timestamp, account_id)
        else:
            result = "Operation not supported at this level."

        results.append(result)

    return results


In [ ]:
queries = [
    ["CREATE_ACCOUNT", "1", "account1"],
    ["CREATE_ACCOUNT", "2", "account1"],
    ["CREATE_ACCOUNT", "3", "account2"],
    ["DEPOSIT", "4", "non-existing", "2700"],
    ["DEPOSIT", "5", "account1", "2700"],
    ["TRANSFER", "6", "account1", "account2", "2701"],
    ["TRANSFER", "7", "account1", "account2", "200"]
]

results = process_queries(queries)
print(results)
#["true", "false", "true", "", "2700", "", "2500"]

['true', 'false', 'true', '', '2700', '', '2500']


In [ ]:
queries = [
    ["CREATE_ACCOUNT", "1", "account3"],
    ["CREATE_ACCOUNT", "2", "account2"],
    ["CREATE_ACCOUNT", "3", "account1"],
    ["DEPOSIT", "4", "account1", "2000"],
    ["DEPOSIT", "5", "account2", "3000"],
    ["DEPOSIT", "6", "account3", "4000"],
    ["TOP_SPENDERS", "7", "3"],
    ["TRANSFER", "8", "account3", "account2", "500"],
    ["TRANSFER", "9", "account3", "account1", "1000"],
    ["TRANSFER", "10", "account1", "account2", "2500"],
    ["TOP_SPENDERS", "11", "3"]
]

results = process_queries(queries)
print(results)
#["true", "true", "true", "2000", "3000", "4000", "account1(0), account2(0), account3(0)", "3500", "2500", "500", "account1(2500), account3(1500), account2(0)"

['true', 'true', 'true', '2000', '3000', '4000', 'account1(0), account2(0), account3(0)', '3500', '2500', '500', 'account1(2500), account3(1500), account2(0)']


In [ ]:
queries = [
    ["CREATE_ACCOUNT", "1", "account1"],
    ["CREATE_ACCOUNT", "2", "account2"],
    ["DEPOSIT", "3", "account1", "2000"],
    ["PAY", "4", "account1", "1000"],
    ["PAY", "100", "account1", "1000"],
    ["GET_PAYMENT_STATUS", "101", "non-existing", "payment1"],
    ["GET_PAYMENT_STATUS", "102", "account2", "payment1"],
    ["GET_PAYMENT_STATUS", "103", "account1", "payment1"],
    ["TOP_SPENDERS", "104", "2"],
    ["DEPOSIT", str(3 + 86400000), "account1", "100"],
    ["GET_PAYMENT_STATUS", str(4 + 86400000), "account1", "payment1"],
    ["DEPOSIT", str(5 + 86400000), "account1", "100"],
    ["DEPOSIT", str(99 + 86400000), "account1", "100"],
    ["DEPOSIT", str(100 + 86400000), "account1", "100"]
    ]

results = process_queries(queries)
print(results)
#Correct answer ["true", "true", "2000", "payment1", "payment2", "", "", "IN_PROGRESS", "account1(2000), account2(0)", "100", "CASHBACK_RECEIVED", "220", "320", "440"]

['true', 'true', '2000', 'payment1', 'payment2', '', '', 'IN_PROGRESS', 'account1(2000), account2(0)', '100', 'CASHBACK_RECEIVED', '220', '320', '440']


In [ ]:

queries = [
["CREATE_ACCOUNT", "1", "account1"],
["CREATE_ACCOUNT", "2", "account2"],
["DEPOSIT", "3", "account1", "2000"],
["DEPOSIT", "4", "account2", "2000"],
["PAY", "5", "account2", "300"],
["TRANSFER", "6", "account1", "account2", "500"],
["MERGE_ACCOUNTS", "7", "account1", "non-existing"],
["MERGE_ACCOUNTS", "8", "account1", "account1"],
["MERGE_ACCOUNTS", "9", "account1", "account2"],
["DEPOSIT", "10", "account1", "100"],
["DEPOSIT", "11", "account2", "100"],
["GET_PAYMENT_STATUS", "12", "account2", "payment1"],
["GET_PAYMENT_STATUS", "13", "account1", "payment1"],
["GET_BALANCE", "14", "account2", "1"],
["GET_BALANCE", "15", "account2", "9"],
["GET_BALANCE", "16", "account1", "11"],
["DEPOSIT", str(5 + 86400000), "account1", "100"]]
results = process_queries(queries)
print(results)

 #["true", "true", "2000", "2000", "payment1", "1500", "false", "false", "true", "3800", "", "", "IN_PROGRESS", "", "", "3800", "3906"]

['true', 'true', '2000', '2000', 'payment1', '1500', 'false', 'false', 'true', '3800', '', '', 'IN_PROGRESS', '', '', '3800', '3906']


In [ ]:
queries = [
["CREATE_ACCOUNT", "1", "account1"],
["DEPOSIT", "2", "account1", "1000"],
["PAY", "3", "account1", "300"],
["GET_BALANCE", "4", "account1", "3"],
["GET_BALANCE", str(5 + 86400000), "account1", str(2 + 86400000)],
["GET_BALANCE", str(6 + 86400000), "account1", str(3 + 86400000)]]
#["true", "1000", "payment1", "700", "700", "706"].
results = process_queries(queries)
print(results)

['true', '1000', 'payment1', '700', '706', '706']
